In [1]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [2]:
df = dd.read_csv("datos_SUBE/Datos_Vega.csv",encoding = "ISO-8859-1", sep=";")

In [3]:
with ProgressBar():
    display(df.head())

[########################################] | 100% Completed |  1.5s


,NROTARJETA,MODO,JURISDICCION,ENTIDAD,LINEA,RAMAL,INTERNO,TIPO_TARIFA,CODIGOCONTRATO,MOTIVO_ATS,SEXO,LONGITUD,LATITUD,FECHA_HORA,ORDEN_TRX
0,7E3D75678B9506DDB9BA6459BC8700,COL,MUNICIPAL,EMPRESA NUEVE DE JULIO S.A.T.,BSAS_LINEA_561B,561BA,0C14D0A20466B9E62B8C,Tarifa_desc_ATSF,621,AUH,F,-58.052,-34.954,04/12/2019 06,4.0
1,NaN,COL,MUNICIPAL,EMPRESA NUEVE DE JULIO S.A.T.,BSAS_LINEA_561B,561BA,0C14D0A20466B9E62B8C,Tarifa_desc_ATSF,621,NaN,M,-58.052,-34.954,04/12/2019 06,NaN
2,NaN,COL,MUNICIPAL,EMPRESA NUEVE DE JULIO S.A.T.,BSAS_LINEA_561B,561BA,0C14D0A20466B9E62B8C,Tarifa_normal,602,NaN,NaN,-58.052,-34.954,04/12/2019 06,NaN
3,NaN,COL,MUNICIPAL,EMPRESA NUEVE DE JULIO S.A.T.,BSAS_LINEA_561B,561BA,0C14D0A20466B9E62B8C,Tarifa_desc_ATSF,621,AUH,F,-58.052,-34.954,04/12/2019 09,NaN
4,NaN,COL,MUNICIPAL,EMPRESA NUEVE DE JULIO S.A.T.,BSAS_LINEA_561B,561BA,0C14D0A20466B9E62B8C,Tarifa_desc_ATSF,621,AUH,F,-58.052,-34.954,04/12/2019 09,NaN


In [5]:
columns_keep = ["LONGITUD","LATITUD","FECHA_HORA","RAMAL","MODO"]
df_fil = df[columns_keep]


cond = "MODO" == "COL"
condition_modo = df_fil["MODO"] == "COL"
df_fil_modo = df_fil[condition_modo]

df_fil1 = df_fil_modo.drop(["MODO"], axis=1)
df_fil1["HORA"] = df_fil1["FECHA_HORA"].apply(lambda x: x[-2:])
df_fil1["FECHA"] = df_fil1["FECHA_HORA"].apply(lambda x: datetime.strptime(x[:-3],"%d/%m/%Y"), meta=('x',str))
df_fil1= df_fil1.drop(["FECHA_HORA"],axis=1)


#### INSPECT SINGLE PARTITION

In [6]:
with ProgressBar():
    df= df_fil1.set_index("FECHA")     
df = df.repartition(freq="1M")


[########################################] | 100% Completed |  1hr  7min 27.7s


ValueError: Partitioning on non-existent column. partition_on=['INDEX'] .columns=['LONGITUD', 'LATITUD', 'RAMAL', 'HORA']

In [7]:
with ProgressBar():
    dd.to_parquet(df,"datos_SUBE/month_sorted", engine="pyarrow", compression="snappy")

[########################################] | 100% Completed |  1hr  5min 13.7s
